In [ ]:
!pip install ultralytics opencv-python-headless

🧠 **Detect and count people**


In [ ]:
# Step 3: Import libs and define processing function
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import csv

# Load YOLOv8n model
model = YOLO('yolo11n.pt')

# Variables to track people and logging
seen_ids = set()
id_log = {}

def process_video(input_path):
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    total_unique_people = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Object tracking with YOLOv8
        results = model.track(
            source=frame,
            persist=True,
            stream=False,
            verbose=False,
            classes=[0]  # Detect only persons
        )

        # Draw boxes, count people, and log IDs
        if results and results[0].boxes.id is not None:
            for box in results[0].boxes:
                class_id = int(box.cls)
                label = results[0].names[class_id]

                if label == 'person':
                    track_id = int(box.id)
                    if track_id not in seen_ids:
                        seen_ids.add(track_id)
                        total_unique_people += 1
                        id_log[track_id] = frame_count

                    # Draw bounding box and label with track ID
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    color = (0, 255, 0)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, f"{label} ID:{track_id}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Display frame every 30 frames
        if frame_count % 30 == 0:
            print(f"Frame {frame_count}, Unique people so far: {total_unique_people}")
            cv2_imshow(frame)

        frame_count += 1

    cap.release()

    print(f"\n✅ Final Count of Unique People: {total_unique_people}")

    # Save log to CSV
    # with open("people_id_log.csv", "w", newline="") as csvfile:
    #     writer = csv.writer(csvfile)
    #     writer.writerow(["Track ID", "First Detected Frame"])
    #     for tid, first_frame in sorted(id_log.items()):
    #         writer.writerow([tid, first_frame])

    # print("📄 'people_id_log.csv' has been saved.")


Step 5: Run the processing

In [ ]:
# Step 2: Upload video file
from google.colab import files
uploaded = files.upload()
input_path = list(uploaded.keys())[0]

In [ ]:
# Step 4: Run the processing
process_video(input_path)